# Solving a realistic task with sklearn

Feel free to skip these theoretical reminders if you feel confident with the content.

### Reminder: loss functions

Training and quality assessment of the model is carried out on independent sets of examples. As a rule, existing examples are divided into two subsets: training (train) and test (test). The choice of split ratio is a compromise. Indeed, the large size of the training leads to better algorithms, but more noise when evaluating the model on the test. Conversely, a large test sample size leads to a less noisy quality assessment, but the trained models are less accurate.

Many classification models predict a rating of membership to a positive class $ \tilde {y} (x) \in R $ (for example, the probability of being a class 1). After that, a decision is made on the class of the object by comparing the estimate with a certain threshold $ \ theta $:

$$y(x) = 
\begin{cases}
+1, &\text{if} \; \tilde{y}(x) \geq \theta \\
-1, &\text{if} \; \tilde{y}(x) < \theta
\end{cases}
$$

In this case, we can consider metrics that can work with the initial response of the classifier. In the assignment, we will work with the AUC-ROC metric, which in this case can be considered as the proportion of incorrectly ordered pairs of objects sorted by ascending the predicted grade 1 grade. A detailed understanding of how the AUC-ROC metrics work for this exercise is not required.

### Reminder: selection of hyperparameters of the model

In machine learning tasks, one should distinguish between model parameters and hyperparameters (structural parameters). Typically, model parameters are adjusted during training (for example, weights in a linear model or a decision tree structure), while hyper parameters are set in advance (for example, the value of the regularization force in a linear model or the maximum depth of a decision tree). Each model, as a rule, has a lot of hyperparameters and there are no universal sets of hyperparameters that work optimally in all tasks, so for each task you need to choose your own set.

To optimize the hyperparameters, models often use __search on the grid (grid search)__: for each hyperparameter, several values are selected, then all combinations of values are selected and a combination is selected on which the model shows the best quality (in terms of the metric being optimized). However, in this case, it is necessary to correctly evaluate the constructed model, namely, to do the partition into a training and test sample. There are several schemes for how this can be implemented:

 - Break the available sample into training and test. In this case, the comparison of a large number of models in the enumeration of hyperparameters leads to a situation where the best model on the test subsample does not retain its qualities on the new data. We can say that there is a _transition_ on a test sample.
 - To eliminate the problem described above, you can split the data into 3 non-overlapping subsamples: training, validation, and test. Validation subsample is used to compare models, and test - for the final quality assessment and comparison of families of models with selected hyperparameters.
 - Another way to compare models is [cross-validation](http://bit.ly/1CHXsNH). There are various cross-validation schemes:
  - Leave-One-Out
  - K-Fold
  - Repeated random sampling
  
Cross validation is computationally expensive, especially if you are iterating over a grid with a very large number of combinations. Given the finiteness of the time to perform the task, a number of compromises arise: 
  - the hyperparameter grid can be made more sparse by going through fewer values of each hyperparameter; however, you should not forget that in this case you can skip a good combination of hyperparameters;
  - cross-validation can be done with a smaller number of splits or folds, but in this case the quality assessment becomes more noisy and the risk of choosing a non-optimal set of hyperparameters increases due to randomness of splitting
  - hyperparameters can be optimized sequentially (greedily) - one by one, and not to go through all the combinations; such a strategy does not always lead to an optimal set;
  - sort through not all combinations of hyperparameters, but a small number of randomly selected ones.

### The task

In this exercise, we will learn to train machine learning models, correctly set up experiments, select hyperparameters, compare and mix models.

The goal to predict the forest cover type (the predominant kind of tree cover) from strictly cartographic variables (as opposed to remotely sensed data). The actual forest cover type for a given 30 x 30 meter cell was determined from US Forest Service (USFS) Region 2 Resource Information System data. Independent variables were then derived from data obtained from the US Geological Survey and USFS. The data is in raw form (not scaled) and contains binary columns of data for qualitative independent variables such as wilderness areas and soil type.

This study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. These areas represent forests with minimal human-caused disturbances, so that existing forest cover types are more a result of ecological processes rather than forest management practices.
 
The study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. Each observation is a 30m x 30m patch. You are asked to predict an integer classification for the forest cover type. The seven types are:

1. Spruce/Fir
2. Lodgepole Pine
3. Ponderosa Pine
4. Cottonwood/Willow
5. Aspen
6. Douglas-fir
7. Krummholz

The training set (11340 observations) contains both features and the Cover_Type.

### Data Fields
__Elevation__ - Elevation in meters  
__Aspect__ - Aspect in degrees azimuth  
__Slope__ - Slope in degrees  
__Horizontal_Distance_To_Hydrology__ - Horz Dist to nearest surface water features  
__Vertical_Distance_To_Hydrology__ - Vert Dist to nearest surface water features  
__Horizontal_Distance_To_Roadways__ - Horz Dist to nearest roadway  
__Hillshade_9am__ (0 to 255 index) - Hillshade index at 9am, summer solstice  
__Hillshade_Noon__ (0 to 255 index) - Hillshade index at noon, summer solstice  
__Hillshade_3pm__ (0 to 255 index) - Hillshade index at 3pm, summer solstice  
__Horizontal_Distance_To_Fire_Points__ - Horz Dist to nearest wildfire ignition points  
__Wilderness_Area__ - Wilderness area designation  
__Soil_Type__ - Soil Type designation  
__Cover_Type__ (7 types, integers 1 to 7) - Forest Cover Type designation  

The wilderness areas are:

1. Rawah Wilderness Area
2. Neota Wilderness Area
3. Comanche Peak Wilderness Area
4. Cache la Poudre Wilderness Area

The soil types are:

1. Cathedral family - Rock outcrop complex, extremely stony.
2. Vanet - Ratake families complex, very stony.
3. Haploborolis - Rock outcrop complex, rubbly.
4. Ratake family - Rock outcrop complex, rubbly.
5. Vanet family - Rock outcrop complex complex, rubbly.
6. Vanet - Wetmore families - Rock outcrop complex, stony.
7. Gothic family.
8. Supervisor - Limber families complex.
9. Troutville family, very stony.
10. Bullwark - Catamount families - Rock outcrop complex, rubbly.
11. Bullwark - Catamount families - Rock land complex, rubbly.
12. Legault family - Rock land complex, stony.
13. Catamount family - Rock land - Bullwark family complex, rubbly.
14. Pachic Argiborolis - Aquolis complex.
15. unspecified in the USFS Soil and ELU Survey.
16. Cryaquolis - Cryoborolis complex.
17. Gateview family - Cryaquolis complex.
18. Rogert family, very stony.
19. Typic Cryaquolis - Borohemists complex.
20. Typic Cryaquepts - Typic Cryaquolls complex.
21. Typic Cryaquolls - Leighcan family, till substratum complex.
22. Leighcan family, till substratum, extremely bouldery.
23. Leighcan family, till substratum - Typic Cryaquolls complex.
24. Leighcan family, extremely stony.
25. Leighcan family, warm, extremely stony.
26. Granile - Catamount families complex, very stony.
27. Leighcan family, warm - Rock outcrop complex, extremely stony.
28. Leighcan family - Rock outcrop complex, extremely stony.
29. Como - Legault families complex, extremely stony.
30. Como family - Rock land - Legault family complex, extremely stony.
31. Leighcan - Catamount families complex, extremely stony.
32. Catamount family - Rock outcrop - Leighcan family complex, extremely stony.
33. Leighcan - Catamount families - Rock outcrop complex, extremely stony.
34. Cryorthents - Rock land complex, extremely stony.
35. Cryumbrepts - Rock outcrop - Cryaquepts complex.
36. Bross family - Rock land - Cryumbrepts complex, extremely stony.
37. Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.
38. Leighcan - Moran families - Cryaquolls complex, extremely stony.
39. Moran family - Cryorthents - Leighcan family complex, extremely stony.
40. Moran family - Cryorthents - Rock land complex, extremely stony.

In [ ]:
# This is a very useful package for dealing with this sort of data
import pandas as pd

In [ ]:
df = pd.read_csv('./forest_train.csv')
df.head()

**Task 1:** Usually, after loading a dataset, some preprocessing is always necessary. In this case, it will be as follows:
 - Save the target variable (the one we want to predict) into a separate variable $y$
 - Save the explanatory variables into a separate variable $X$

In [ ]:
df.columns

In [ ]:
y = np.array(df.Cover_Type)

X = # Select all the columns except the target here.
X.head()

 - In this part we will work on binary classification problem. That is why we will select only the examples with Cover Type 1 and 2. Next we will classify examples with Cover Type 1 vs examples with Cover Type 2.

In [ ]:
X_bin = X[(y == 1) | (y == 2)]
y_bin = y[(y == 1) | (y == 2)]

 - Note that not all features are numeric. In the beginning we will work only with numeric features. Save them separately.

In [ ]:
numeric_cols = [
# Select only the columns that are numerical here.
]

num_features = X_bin[numeric_cols] # Select only numeric features here
num_features.head()

### Training classifiers on real features

In this section, it will be necessary to work only with real attributes and the target variable.

In the beginning we will look at how the selection of hyperparameters on the grid works and how the quality of the partitioning affects the quality. Now and further we will consider 4 algorithms:
 - [kNN](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
 - [DecisonTree](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
 - [SGD Linear Classifier](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)
 - [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

To begin with, the first three algorithms will choose one hyperparameter, which we will optimize:
  - kNN - number of neighbors (**n_neighbors**)
  - DecisonTree - tree depth (**max_depth**)
  - SGD Linear Classifier - optimized function (**loss**)
 
Leave the default values for the remaining hyperparameters. For the selection of hyperparameters, use the search on the grid, which is implemented in the class [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV). As a cross-validation scheme, use 5-Fold CV, which can be set using the class [KFoldCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

![](https://i.stack.imgur.com/YWgro.gif)



**Task 2:** For each algorithm, select the optimal values of the specified hyperparameters, to do so select the values that should be tested in the grid search.

Then we build a graph of the average value of the quality of the cross-validation algorithm for a given value of the hyperparameter, which also display the confidence interval.

To obtain the value of quality on each fold, the average value of quality and other useful information, we use the field [*cv results_*](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)

Which algorithm has the highest average quality?

Largest confidence interval?

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Insert the values that should be checked during the grid search of optimal parameters
classifiers = {
    KNeighborsClassifier:   [{'n_neighbors': [<COMPLETE HERE>]}],
    DecisionTreeClassifier: [{'max_depth'  : [<COMPLETE HERE>]}],
    SGDClassifier:          [{'loss'       : [<COMPLETE HERE>]}]
}

trained_clfs = []

for classifier, params in classifiers.items():
    clf = GridSearchCV(classifier(), param_grid=params, cv=KFold(n_splits=5), return_train_score=True)
    clf.fit(num_features.values, y_bin)
    trained_clfs.append([clf])

In [ ]:
plt.figure(figsize=fig_size)
mean_values = []
stds = []
names = []

for clf in trained_clfs:
    mean_vals = clf[0].cv_results_['mean_test_score']
    std_vals = clf[0].cv_results_['std_test_score']
    params = clf[0].cv_results_['params'][0]
    
    idx = np.argmin(clf[0].cv_results_['rank_test_score'])
    opt_params = clf[0].cv_results_['params'][idx]
    mean = clf[0].cv_results_['mean_test_score'][idx]
    std = clf[0].cv_results_['std_test_score'][idx]
    name = clf[0].estimator.__str__().split('(')[0] + ' , {}'.format(opt_params)
    stds.append(std)
    mean_values.append(mean)
    names.append(name)

plt.title('Performance of classifiers with the best optimized parameter value')
plt.errorbar(np.arange(len(mean_values)), mean_values, stds, linestyle='None', marker='*')
plt.ylabel('accuracy')
plt.xlabel('classifier')
plt.xticks(np.arange(len(mean_values)), names)
plt.show()

**Task 3:** Now let's select the number of trees (**n_estimators**) in the RandomForest algorithm. As you might know, in general, Random Forest is less likely to overfit with an increase in the number of trees. Pick the number of trees from which the quality on cross-validation stabilizes. Note that to conduct this experiment, it is not necessary to train many random forests with different numbers of trees from scratch: train one random forest with the maximum interesting number of trees, and then consider subsets of trees of different sizes consisting of trees of the constructed forest (field [**estimators_**](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)). In further experiments, use the number of trees found.

Fill in the following code.

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
kf = KFold(n_splits=5)
clfs = []

for train_index, test_index in kf.split(num_features.values):
    X_train, X_test = num_features.values[train_index], num_features.values[test_index]
    y_train, y_test = y_bin[train_index], y_bin[test_index]
    
    clf = #COMPLETE HERE: Create random forest classifier with 300 trees
    #COMPLETE HERE: Train random forest classifier here
    
    clfs.append([clf, X_test, y_test - 1])

In [ ]:
scores_by_fold = []

for clf_n_data in clfs:
    clf, x, y = clf_n_data
    predictions = []
    
    for estimator in clf.estimators_:
        predictions.append(<Make prediction here>)
        
    scores_by_fold.append([np.array(predictions), y])
    
quality_by_num_trees = []
for num_trees in range(1, 301):
    quality = []
    
    for predictions, labels in scores_by_fold:
        vote = (predictions[:num_trees, :].mean(axis=0) > 0.5).astype(np.int64)
        quality.append((vote == labels).sum() / float(len(labels)))
        
    quality_by_num_trees.append(quality)

In [ ]:
means = []
stds = []
xs = []
for i, num_trees in enumerate(range(1, 301)):
    means.append(np.mean(quality_by_num_trees[i]))
    stds.append(np.std(quality_by_num_trees[i]))
    xs.append(num_trees)
    
plt.figure(figsize=fig_size)
plt.plot(xs, means, 'b', label='accuracy')
plt.title('Avg quality by num_trees')
plt.xlabel('num_trees')
plt.xlabel('accuracy')
plt.legend(loc="best")
plt.fill_between(xs, np.array(means) - np.array(stds), np.array(means) + np.array(stds), facecolor='green')

When learning algorithms, it is worth paying attention not only to their quality, but also how they deal with the data. In this problem, it turned out that some of the algorithms used are sensitive to the scale of features. To make sure that this could affect the quality, let's look at the meaning of the attributes themselves.

**Task 4:** Look at the values of the features **Aspect**, **Slope**, **Hillshade_9am**. How is the data distributed? Which of the considered algorithms can be affected by this? Can scaling of the features affect these algorithms?

In [ ]:
plt.figure(figsize=fig_size)
plt.hist(np.array(num_features.Elevation))
plt.yscale('log')
plt.ylabel('freqs')
plt.xlabel('Elevation')
plt.title('Elevation feature values distribution')
plt.show()

In [ ]:
# Create the same graph for the Aspect, Slope and Hillshade_9am features

You can scale attributes, for example, in one of the following ways:
 - $x_{new} = \dfrac{x - \mu}{\sigma}$, where $\mu, \sigma$ — the mean and standard deviation of the characteristic value over the entire sample (see the function [scale](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html))
 - $x_{new} = \dfrac{x - x_{min}}{x_{max} - x_{min}}$, where $[x_{min}, x_{max}]$ — minimum interval of characteristic values

Similar scaling schemes are given in classes [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) and [MinMaxScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler).
 
**Task 5:** Scale all numerical attributes using the standard scaler and select the optimal values of the hyperparameters in the same way as above.

Has the quality of some algorithms changed and why?

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaled_features = #COMPLETE HERE: Scale features

In [ ]:
#Compute and plot average quality of the first three models with the normalised features 
#As well as the quality of the random forect classifier by num trees with the normalised features 
#You can reuse the previous code

**Task 6:** Now go through several hyperparameters across the grid and find the optimal combinations (best average quality value) for each algorithm in this case:
  - KNN - number of neighbors (**n_neighbors**) and metric (**metric**)
  - DecisonTree - tree depth (**max_depth**) and partitioning criterion (**criterion**)
  - RandomForest - the partitioning criterion in trees (**criterion**) and the maximum number of considered features (**max_features**); use the trees found earlier
  - SGDClassifier - optimized function (**loss**) and **penalty**
 
Please note that this operation can be resource intensive and time consuming. How to optimize the selection of parameters on the grid is described in the section "Selection of hyperparameters of the model".

Which algorithm has the best quality?

In [ ]:
# Insert values that should be checked during the grid search of optimal parameters

classifiers = {
    KNeighborsClassifier: [{'n_neighbors': [<COMPLETE HERE>],
                            'metric': [<COMPLETE HERE>]
                           }],
    DecisionTreeClassifier: [{'max_depth': [<COMPLETE HERE>], 
                              'criterion': [<COMPLETE HERE>]
                             }],
    SGDClassifier: [{'loss': [<COMPLETE HERE>],
                     'penalty': [<COMPLETE HERE>]}],
    RandomForestClassifier: [{'criterion': [<COMPLETE HERE>],
                              'max_features': [<COMPLETE HERE>],
                              'n_estimators': [<COMPLETE HERE>]
                             }]
}

trained_clfs = []

for classifier, params in classifiers.items():
    clf = GridSearchCV(classifier(), param_grid=params, cv=KFold(n_splits=5), return_train_score=True)
    clf.fit(scaled_features, y_bin)
    trained_clfs.append([clf, ])

In [ ]:
plt.figure(figsize=fig_size)
mean_values = []
stds = []
names = []

for clf in trained_clfs:
    mean_vals = clf[0].cv_results_['mean_test_score']
    std_vals = clf[0].cv_results_['std_test_score']
    params = clf[0].cv_results_['params'][0]
    
    idx = np.argmin(clf[0].cv_results_['rank_test_score'])
    opt_params = clf[0].cv_results_['params'][idx]
    mean = clf[0].cv_results_['mean_test_score'][idx]
    std = clf[0].cv_results_['std_test_score'][idx]
    name = clf[0].estimator.__str__().split('(')[0]
    stds.append(std)
    mean_values.append(mean)
    names.append(name)

plt.title('Performance of classifiers with the best optimized parameter value')
plt.errorbar(np.arange(len(mean_values)), mean_values, stds, linestyle='None', marker='*')
plt.ylabel('accuracy')
plt.xlabel('classifier')
plt.xticks(np.arange(len(mean_values)), names)
plt.show()

**Task 7:** We will create [learning curves](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html) for the different algorithms, depicting the dependence of the quality on test and training samples on the number of objects on which models are trained. Look at the behavior of the curves and answer the questions:
* Can the quality on the test sample decrease with an increase in the number of objects? And on the training? Why? 
* For what purposes can quality knowledge be used on the training part of the sample?
* Which algorithm is better trained on fewer objects?
* Can the addition of new objects significantly improve the quality of any of the algorithms or, with the existing data set for all algorithms, saturation occur?

In [ ]:
from sklearn.model_selection import learning_curve

In [ ]:
# Taken from here: http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure(figsize=fig_size)
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
for clf in trained_clfs:
    name = clf[0].estimator.__str__().split('(')[0]
    plot_learning_curve(clf[0].best_estimator_, 'Learning Curves ({})'.format(name), scaled_features, y_bin, cv=KFold(n_splits=5))
    plt.show()

### Adding categorical features to models

So far we have not used non-numeric attributes that are in dataset. Let's see if we did the right thing and whether the quality of the models will increase after adding these attributes.

**Task 8:** Select all categorical features to be transformed using the one-hot-encoding method ([pandas.get_dummies](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) or [DictVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html) / [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) from sklearn).

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [ ]:
X.columns

In [ ]:
cat_columns = [
# Select only the columns that are categorical here.
]

cat_features = X_bin[cat_columns]
cat_features.head()

In [ ]:
enc = OneHotEncoder(categories='auto')
categorical_encoded_features = enc.fit_transform(cat_features).todense()

Since the features turned out to be quite numerous, in this work we will not re-select the optimal hyperparameters for models taking into account new signs (although it would be better to do it). 

We add the coded categorical to the scaled numerical features. 

**Task 9:** Train the algorithms with the best hyper-parameters found earlier. Did the addition of new features increase the quality? Measure quality as before using a 5-fold CV. For this it is convenient to use the function [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.cross_val_score.html#sklearn.cross_validation.cross_val_score).

Is the best classifier now different than before?

In [ ]:
all_features = np.hstack([X_bin, categorical_encoded_features])

In [ ]:
# Train the classifiers here

### Mixing models (blending)


In all the preceding paragraphs, we obtained many strong models that can be quite different in nature (for example, the method of the nearest neighbors and the random forest). Often in practice it is possible to increase the quality of prediction by mixing different models. Let's see if this approach really gives an increase in quality.

Choose from the constructed models of the two previous points two, which gave the highest quality on cross-validation (we denote them $ {clf}_1 $ and $ {clf}_2 $). Next, build a new classifier, whose answer on some object $ x $ will look like this:

$$result(x) = {clf}_1(x) * \alpha + {clf}_2(x) * (1 - \alpha)$$

where $ \alpha $ is a hyper-parameter of the new classifier.

**Task 10:**
When implementing models, it is good practice to create sklearn-compatible classes. First, such an implementation will have a standard interface and will allow other people to train the models you have implemented easily. Secondly, it is possible to use any sklearn package functionality that accepts a model as input, for example, the class **GridSearchCV**, **learning_curve** and others.

Create a classifier that is initialized with two arbitrary classifiers and the $ \ alpha $ parameter. During training, such a classifier should train both basic models, and at the stage of prediction, knead predictions of basic models according to the formula indicated above.

To create a custom classifier, you must inherit from the base classes:
*[BaseEstimator](http://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html), [ClassifierMixin](http://scikit-learn.org/stable/modules/generated/sklearn.base.ClassifierMixin.html)* and implement methods*\_\_init\_\_, fit, predict and predict_proba*. Example sklearn-compatible classifier with comments can be found [here](http://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator) 

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
class BlendingClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, clf1=None, clf2=None, alpha=None):
        pass # IMPLEMENT ME
    
    def fit(self, x, y):
        pass # IMPLEMENT ME
    
    def predict(self, x):
        pass # IMPLEMENT ME
    
    def predict_proba(self, x):
        pass # IMPLEMENT ME

**Task 11:** Select the value $ \alpha $ for this classifier from the grid from 0 to 1. If the class is implemented correctly, then you can use *GridSearchCV*, as is the case with conventional classifiers.

Did this approach increase in quality compared to models that were trained separately? Explain why even simple blending of models can influence the final quality?

In [ ]:
clf = GridSearchCV(
    BlendingClassifier(
        clf1=trained_clfs[1][0],
        clf2=trained_clfs[3][0]
    ), param_grid=[{'alpha': np.linspace(0, 1, num=10)}],
    cv=KFold(n_splits=5), return_train_score=True)

trained = clf.fit(all_features, y_bin)

In [ ]:
plt.figure(figsize=fig_size)
plt.plot(trained.cv_results_['param_alpha'].data.astype(np.float32), trained.cv_results_['mean_test_score'], 'b', label='accuracy')
plt.title('Mean folds accuracy')
plt.xlabel('x')
plt.xlabel('alpha')
plt.legend(loc="best")
plt.fill_between(trained.cv_results_['param_alpha'].data.astype(np.float32), trained.cv_results_['mean_test_score'] - trained.cv_results_['std_test_score'], trained.cv_results_['mean_test_score'] +  trained.cv_results_['std_test_score'], facecolor='green')

In [ ]:
trained_clfs.append([trained])

## Models comparison

After many models have been built, we have to compare them with each other. 

**Task 12:** Reflect back on what we did and draw conclusions about the classifiers in terms of how they dealt with the different features and the complexity of the model itself (what kind of hyperparameters the model has, whether changing the value of the hyperparameter greatly affects the quality of the model).

In [ ]:
mean_values = []
splits = []
stds = []
names = []

for clf in trained_clfs:
    mean_vals = clf[0].cv_results_['mean_test_score']
    std_vals = clf[0].cv_results_['std_test_score']
    params = clf[0].cv_results_['params'][0]
    
    idx = np.argmin(clf[0].cv_results_['rank_test_score'])
    opt_params = clf[0].cv_results_['params'][idx]
    mean = clf[0].cv_results_['mean_test_score'][idx]
    std = clf[0].cv_results_['std_test_score'][idx]
    name = clf[0].estimator.__str__().split('(')[0]
    split = []
    for i in range(5):
        split.append(clf[0].cv_results_['split{}_test_score'.format(i)][idx])
    stds.append(std)
    mean_values.append(mean)
    names.append(name)
    splits.append(split)

In [ ]:
plt.figure(figsize=fig_size)
results_df = pd.DataFrame(data=np.asarray(splits).T, columns=[names])
ax = results_df.boxplot(figsize=(15, 8), return_type='axes', sym='k.')
_ = plt.setp(ax.lines, linewidth=2.0)
plt.show()

# Bonus task 
Try to find the best classifier (using the procedure above) on the full dataset with all cover types. 
You are free to create derived features from existing ones, experiment with different methods (classes of sklearn) and even to apply deep methods here.
You are not allowed to use additional data.
You are allowed to use publically available code.